# Procesamiento de Elecciones Presidenciales - Manzanillo (2012-2024)
## Objetivo
El propósito de este notebook es limpiar, estandarizar y estructurar los resultados electorales para las elecciones Presidenciales en Manzanillo, abarcando el periodo de 2012 a 2024. El objetivo final es generar archivos de datos limpios y consistentes que puedan ser fácilmente consolidados en una única base de datos para su posterior análisis y visualización geoespacial.

### Manzanillo Presidente 2012

In [18]:
import pandas as pd
import numpy as np
import os

# --- 1. CONFIGURACIÓN DE RUTAS ---
BASE_DIR = '/Users/omartellez/Manzanillo' 

# CAMBIO: Apuntamos a la nueva carpeta
input_folder = os.path.join(BASE_DIR, '01_datos_brutos', 'Presidente')
output_folder = os.path.join(BASE_DIR, '02_datos_limpios', 'Presidente')

input_file = 'Manzanillo_Presidente_2012.csv'
output_file = 'Manzanillo_Presidente_2012_limpio.csv'

# --- 2. CARGA DE DATOS ---
file_path = os.path.join(input_folder, input_file)
# Usamos 'utf-8-sig' para manejar la marca BOM (ï»¿)
df = pd.read_csv(file_path, encoding='utf-8-sig')
print("Datos originales de presidente 2012 cargados.")

df.info()

Datos originales de presidente 2012 cargados.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   CIRCUNSCRIPCION     200 non-null    int64 
 1   ID_ESTADO           200 non-null    int64 
 2   NOMBRE_ESTADO       200 non-null    object
 3   ID_DISTRITO         200 non-null    int64 
 4   CABECERA_DISTRITAL  200 non-null    object
 5   ID_MUNICIPIO        200 non-null    int64 
 6   MUNICIPIO           200 non-null    object
 7   SECCION             200 non-null    int64 
 8   CASILLA             200 non-null    object
 9   PAN                 200 non-null    int64 
 10  PRI                 200 non-null    int64 
 11  PRD                 200 non-null    int64 
 12  PVEM                200 non-null    int64 
 13  PT                  200 non-null    int64 
 14  MC                  200 non-null    int64 
 15  NVA_ALIANZA         200 non-

,CIRCUNSCRIPCION,ID_ESTADO,NOMBRE_ESTADO,ID_DISTRITO,CABECERA_DISTRITAL,ID_MUNICIPIO,MUNICIPIO,SECCION,CASILLA,PAN,...,PRD_MC,PT_MC,NUM_VOTOS_CAN_NREG,NUM_VOTOS_NULOS,TOTAL_VOTOS,LISTA_NOMINAL,ESTATUS_ACTA,TEPJF,OBSERVACIONES,RUTA_ACTA
0,5,6,COLIMA,2,MANZANILLO,8,MANZANILLO,200,B,120,...,2,0,0,7,411,720,Acta casilla,NaN,NaN,JE2012_06_02_0200_1_B_0_2.jpg
1,5,6,COLIMA,2,MANZANILLO,8,MANZANILLO,201,B,33,...,0,1,0,2,97,136,Grupo de recuento,NaN,NaN,JE2012_06_02_1_GPO1.pdf
2,5,6,COLIMA,2,MANZANILLO,8,MANZANILLO,202,B,138,...,1,0,0,7,367,594,Grupo de recuento,NaN,NaN,JE2012_06_02_1_GPO1.pdf
3,5,6,COLIMA,2,MANZANILLO,8,MANZANILLO,202,C01,115,...,0,1,0,13,375,594,Reservada para el consejo,NaN,NaN,JE2012_06_02_1_VR.pdf
4,5,6,COLIMA,2,MANZANILLO,8,MANZANILLO,202,C02,104,...,3,0,0,9,347,593,Acta casilla,NaN,NaN,JE2012_06_02_0202_2_C_0_2.jpg


In [22]:
# --- 3. LIMPIEZA Y ESTRUCTURACIÓN ---
df_mapped = df.copy()

# a) Estandarizar todos los nombres de columna a minúsculas
df_mapped.columns = df_mapped.columns.str.lower()

# b) Eliminar columnas de contexto y casi vacías
columnas_a_eliminar = [
    'circunscripcion', 'id_estado', 'nombre_estado', 'id_distrito', 
    'cabecera_distrital', 'id_municipio', 'municipio', 'estatus_acta', 
    'tepjf', 'observaciones', 'ruta_acta'
]
df_mapped = df_mapped.drop(columns=columnas_a_eliminar, errors='ignore')

# c) Renombrar columnas a nuestro formato estándar
mapa_renombre = {
    'casilla': 'casilla',
    'seccion': 'seccion',
    'nva_alianza': 'na',
    'num_votos_can_nreg': 'voto_no_registrado',
    'num_votos_nulos': 'voto_nulo',
    'total_votos': 'voto_total',
    'lista_nominal': 'lista_nominal'
}
df_mapped = df_mapped.rename(columns=mapa_renombre)

# --- 4. CÁLCULO DE MÉTRICAS ANALÍTICAS (MODELO COMPLETO) ---
# a) Definir competidores y bloques para 2012 (idénticos a Dip. Fed.)
competidores_2012 = [
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'na',
    'pri_pvem', 'prd_pt_mc', 'prd_pt', 'prd_mc', 'pt_mc'
]
proxy_morena_2012 = ['prd', 'pt', 'mc', 'prd_pt_mc', 'prd_pt', 'prd_mc', 'pt_mc']
oposicion_2012 = ['pan', 'pri', 'pvem', 'na', 'pri_pvem']

# b) Cálculo de Bloques (Análisis Macro)
df_mapped['morena_coalitions'] = df_mapped[proxy_morena_2012].sum(axis=1)
df_mapped['non_morena_coalitions'] = df_mapped[oposicion_2012].sum(axis=1)

# c) Composición del Voto "Morena" (Proxy 2012)
df_mapped['voto_morena_solo'] = 0
df_mapped['voto_morena_aliados'] = df_mapped[proxy_morena_2012].sum(axis=1)

# d) Ganador por Casilla
df_competidores = df_mapped[competidores_2012]
df_mapped['ganador_casilla_partido'] = df_competidores.idxmax(axis=1)
df_mapped['ganador_casilla_votos'] = df_competidores.max(axis=1)

# e) Cálculo del Segundo Lugar y Margen
segundo_lugar_votos = np.sort(df_competidores.values, axis=1)[:, -2]
df_mapped['segundo_lugar_votos'] = segundo_lugar_votos
temp_df = df_competidores.copy()
for index, row in temp_df.iterrows():
    winner_col = df_mapped.loc[index, 'ganador_casilla_partido']
    if pd.notna(winner_col) and winner_col in temp_df.columns:
        temp_df.loc[index, winner_col] = np.nan
df_mapped['segundo_lugar_partido'] = temp_df.idxmax(axis=1)
df_mapped['margen_victoria'] = df_mapped['ganador_casilla_votos'] - df_mapped['segundo_lugar_votos']

# --- 5. ORGANIZACIÓN FINAL Y GUARDADO ---
columnas_finales = [
    'seccion', 'casilla', 'lista_nominal',
    # Votos crudos
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'na', 'pri_pvem', 
    'prd_pt_mc', 'prd_pt', 'prd_mc', 'pt_mc',
    # Analíticas
    'morena_coalitions', 'non_morena_coalitions', 'voto_morena_solo', 'voto_morena_aliados',
    'ganador_casilla_partido', 'ganador_casilla_votos', 'segundo_lugar_partido', 
    'segundo_lugar_votos', 'margen_victoria',
    # Cierre
    'voto_no_registrado', 'voto_nulo', 'voto_total'
]
df_mapped = df_mapped[columnas_finales]

print("\nDatos limpios de 2012 (Presidente) con el modelo de análisis completo:")
df_mapped.info()

# Guardar el archivo limpio
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, output_file)
df_mapped.to_csv(output_path, index=False)
print(f"\nArchivo limpio y actualizado de 2012 (Presidente) guardado en: {output_path}")



Datos limpios de 2012 (Presidente) con el modelo de análisis completo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   seccion                  200 non-null    int64 
 1   casilla                  200 non-null    object
 2   lista_nominal            200 non-null    int64 
 3   pan                      200 non-null    int64 
 4   pri                      200 non-null    int64 
 5   prd                      200 non-null    int64 
 6   pvem                     200 non-null    int64 
 7   pt                       200 non-null    int64 
 8   mc                       200 non-null    int64 
 9   na                       200 non-null    int64 
 10  pri_pvem                 200 non-null    int64 
 11  prd_pt_mc                200 non-null    int64 
 12  prd_pt                   200 non-null    int64 
 13  prd_mc                 

### Manzanillo Presidente 2018

In [26]:
# --- 1. CONFIGURACIÓN DE RUTAS ---
BASE_DIR = '/Users/omartellez/Manzanillo' 

# CAMBIO: Apuntamos a la nueva carpeta
input_folder = os.path.join(BASE_DIR, '01_datos_brutos', 'Presidente')
output_folder = os.path.join(BASE_DIR, '02_datos_limpios', 'Presidente')

input_file = 'Manzanillo_Presidente_2018.csv'
output_file = 'Manzanillo_Presidente_2018_limpio.csv'

# --- 2. CARGA DE DATOS ---
file_path = os.path.join(input_folder, input_file)
# Usamos 'utf-8-sig' para manejar la marca BOM (ï»¿)
df = pd.read_csv(file_path, encoding='utf-8-sig')
print("Datos originales de presidente 2018 cargados.")

df.info()
df.head(5)

Datos originales de presidente 2018 cargados.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Data columns (total 43 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CIRCUNSCRIPCION     233 non-null    int64  
 1   ID_ESTADO           233 non-null    int64  
 2   NOMBRE_ESTADO       233 non-null    object 
 3   ID_DISTRITO         233 non-null    int64  
 4   CABECERA_DISTRITAL  233 non-null    object 
 5   ID_MUNICIPIO        233 non-null    int64  
 6   MUNICIPIO           233 non-null    object 
 7   SECCION             233 non-null    int64  
 8   TIPO_CASILLA        233 non-null    object 
 9   ID_CASILLA          233 non-null    int64  
 10  EXT_CONTIGUA        233 non-null    int64  
 11  CASILLA             233 non-null    object 
 12  PAN                 233 non-null    int64  
 13  PRI                 233 non-null    int64  
 14  PRD                 233 non-null    int64  
 15  PVEM       

,CIRCUNSCRIPCION,ID_ESTADO,NOMBRE_ESTADO,ID_DISTRITO,CABECERA_DISTRITAL,ID_MUNICIPIO,MUNICIPIO,SECCION,TIPO_CASILLA,ID_CASILLA,...,CAND_IND1,CAND_IND2,NUM_VOTOS_CAN_NREG,NUM_VOTOS_NULOS,TOTAL_VOTOS,LISTA_NOMINAL,ESTATUS_ACTA,TRIBUNAL,OBSERVACIONES,RUTA_ACTA
0,5,6,COLIMA,2,MANZANILLO,8,MANZANILLO,200,B,1,...,1,20,0,6,381,606,Recuento,NaN,NaN,JE2018_06_02_PRE_GPO1P.pdf
1,5,6,COLIMA,2,MANZANILLO,8,MANZANILLO,201,B,1,...,0,1,0,2,77,123,Recuento,NaN,NaN,JE2018_06_02_PRE_GPO1P.pdf
2,5,6,COLIMA,2,MANZANILLO,8,MANZANILLO,202,B,1,...,3,24,0,6,358,632,Recuento,NaN,NaN,JE2018_06_02_PRE_VR.pdf
3,5,6,COLIMA,2,MANZANILLO,8,MANZANILLO,202,C,1,...,0,12,0,18,381,632,Cotejo,NaN,NaN,JE2018_06_02_PRE_202_1_C_0_12977.jpg
4,5,6,COLIMA,2,MANZANILLO,8,MANZANILLO,202,C,2,...,0,22,0,10,384,632,Recuento,NaN,NaN,JE2018_06_02_PRE_VR.pdf


In [28]:
# --- 3. LIMPIEZA Y ESTRUCTURACIÓN ---
df_mapped = df.copy()

# a) Estandarizar todos los nombres de columna a minúsculas
df_mapped.columns = df_mapped.columns.str.lower()

# b) Eliminar columnas de contexto y vacías
columnas_a_eliminar = [
    'circunscripcion', 'id_estado', 'nombre_estado', 'id_distrito', 
    'cabecera_distrital', 'id_municipio', 'municipio', 'id_casilla', 
    'ext_contigua', 'tipo_casilla', 'estatus_acta', 'tribunal', 
    'observaciones', 'ruta_acta'
]
df_mapped = df_mapped.drop(columns=columnas_a_eliminar, errors='ignore')

# c) Renombrar columnas a nuestro formato estándar
mapa_renombre = {
    'casilla': 'casilla',
    'seccion': 'seccion',
    'na': 'na',
    'morena': 'morena',
    'es': 'es',
    'cand_ind1': 'independiente_1',
    'cand_ind2': 'independiente_2',
    'num_votos_can_nreg': 'voto_no_registrado',
    'num_votos_nulos': 'voto_nulo',
    'total_votos': 'voto_total',
    'lista_nominal': 'lista_nominal'
}
df_mapped = df_mapped.rename(columns=mapa_renombre)

# --- 4. CÁLCULO DE MÉTRICAS ANALÍTICAS (MODELO COMPLETO) ---
# a) Definir competidores y bloques para 2018
competidores_2018 = [
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'na', 'morena', 'es',
    'pan_prd_mc', 'pan_prd', 'pan_mc', 'prd_mc',
    'pri_pvem_na', 'pri_pvem', 'pri_na', 'pvem_na',
    'pt_morena_es', 'pt_morena', 'pt_es', 'morena_es',
    'independiente_1', 'independiente_2'
]
competidores_2018 = [col for col in competidores_2018 if col in df_mapped.columns]

partidos_morena_2018_total = [col for col in ['morena', 'pt', 'es', 'pt_morena_es', 'pt_morena', 'pt_es', 'morena_es'] if col in df_mapped.columns]
partidos_morena_2018_aliados = [col for col in ['pt', 'es', 'pt_morena_es', 'pt_morena', 'pt_es', 'morena_es'] if col in df_mapped.columns]
oposicion_2018 = [col for col in competidores_2018 if col not in partidos_morena_2018_total]

# b) Cálculo de Bloques (Análisis Macro)
df_mapped['morena_coalitions'] = df_mapped[partidos_morena_2018_total].sum(axis=1)
df_mapped['non_morena_coalitions'] = df_mapped[oposicion_2018].sum(axis=1)

# c) Composición del Voto "Morena"
df_mapped['voto_morena_solo'] = df_mapped['morena']
df_mapped['voto_morena_aliados'] = df_mapped[partidos_morena_2018_aliados].sum(axis=1)

# d) Ganador por Casilla
df_competidores = df_mapped[competidores_2018]
df_mapped['ganador_casilla_partido'] = df_competidores.idxmax(axis=1)
df_mapped['ganador_casilla_votos'] = df_competidores.max(axis=1)

# e) Cálculo del Segundo Lugar y Margen
segundo_lugar_votos = np.sort(df_competidores.values, axis=1)[:, -2]
df_mapped['segundo_lugar_votos'] = segundo_lugar_votos
temp_df = df_competidores.copy()
for index, row in temp_df.iterrows():
    winner_col = df_mapped.loc[index, 'ganador_casilla_partido']
    if pd.notna(winner_col) and winner_col in temp_df.columns:
        temp_df.loc[index, winner_col] = np.nan
df_mapped['segundo_lugar_partido'] = temp_df.idxmax(axis=1)
df_mapped['margen_victoria'] = df_mapped['ganador_casilla_votos'] - df_mapped['segundo_lugar_votos']

# --- 5. ORGANIZACIÓN FINAL Y GUARDADO ---
columnas_finales = [
    'seccion', 'casilla', 'lista_nominal',
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'na', 'morena', 'es',
    'pan_prd_mc', 'pan_prd', 'pan_mc', 'prd_mc',
    'pri_pvem_na', 'pri_pvem', 'pri_na', 'pvem_na',
    'pt_morena_es', 'pt_morena', 'pt_es', 'morena_es',
    'independiente_1', 'independiente_2',
    'morena_coalitions', 'non_morena_coalitions', 'voto_morena_solo', 'voto_morena_aliados',
    'ganador_casilla_partido', 'ganador_casilla_votos', 'segundo_lugar_partido', 
    'segundo_lugar_votos', 'margen_victoria',
    'voto_no_registrado', 'voto_nulo', 'voto_total'
]
df_mapped = df_mapped[columnas_finales]

print("\nDatos limpios de 2018 (Presidente) con el modelo de análisis completo:")
df_mapped.info()

# Guardar el archivo limpio
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, output_file)
df_mapped.to_csv(output_path, index=False)
print(f"\nArchivo limpio y actualizado de 2018 (Presidente) guardado en: {output_path}")


Datos limpios de 2018 (Presidente) con el modelo de análisis completo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Data columns (total 38 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   seccion                  233 non-null    int64 
 1   casilla                  233 non-null    object
 2   lista_nominal            233 non-null    int64 
 3   pan                      233 non-null    int64 
 4   pri                      233 non-null    int64 
 5   prd                      233 non-null    int64 
 6   pvem                     233 non-null    int64 
 7   pt                       233 non-null    int64 
 8   mc                       233 non-null    int64 
 9   na                       233 non-null    int64 
 10  morena                   233 non-null    int64 
 11  es                       233 non-null    int64 
 12  pan_prd_mc               233 non-null    int64 
 13  pan_prd                

### Manzanillo PRESIDENTE 2024

In [37]:
input_file = "Manzanillo_PRESIDENTE_2024.csv"
output_file = 'Manzanillo_PRESIDENTE_2024_limpio.csv'

file_path = os.path.join(input_folder, input_file)
df = pd.read_csv(file_path)

print("Datos originales:")
print(df.info())
df.head(5)

Datos originales:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 34 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   CLAVE_CASILLA                217 non-null    object
 1   CLAVE_ACTA                   217 non-null    object
 2   ID_ENTIDAD                   217 non-null    int64 
 3   ENTIDAD                      217 non-null    object
 4   ID_DISTRITO_FEDERAL          217 non-null    int64 
 5   DISTRITO_FEDERAL             217 non-null    object
 6   SECCION                      217 non-null    int64 
 7   ID_CASILLA                   217 non-null    int64 
 8   TIPO_CASILLA                 217 non-null    object
 9   EXT_CONTIGUA                 217 non-null    int64 
 10  CASILLA                      217 non-null    object
 11  TIPO_ACTA                    217 non-null    object
 12  PAN                          217 non-null    int64 
 13  PRI              

,CLAVE_CASILLA,CLAVE_ACTA,ID_ENTIDAD,ENTIDAD,ID_DISTRITO_FEDERAL,DISTRITO_FEDERAL,SECCION,ID_CASILLA,TIPO_CASILLA,EXT_CONTIGUA,...,PVEM_PT,PVEM_MORENA,PT_MORENA,CANDIDATO/A NO REGISTRADO/A,VOTOS NULOS,TOTAL_VOTOS_CALCULADOS,LISTA_NOMINAL,OBSERVACIONES,MECANISMOS_TRASLADO,FECHA_HORA
0,'060200B0100','060200B01002',6,COLIMA,2,VALLE DE LAS GARZAS,200,1,B,0,...,0,0,7,0,7,384,673,Recuento (Para recuento-SRA),I,05/06/24 15:00
1,'060202B0100','060202B01002',6,COLIMA,2,VALLE DE LAS GARZAS,202,1,B,0,...,0,4,2,1,9,326,591,Recuento (Para recuento-SRA),I,05/06/24 15:52
2,'060202C0100','060202C01002',6,COLIMA,2,VALLE DE LAS GARZAS,202,1,C,0,...,1,4,5,0,3,333,591,Cotejo (Levantada en Casilla),I,05/06/24 11:44
3,'060202C0200','060202C02002',6,COLIMA,2,VALLE DE LAS GARZAS,202,2,C,0,...,1,3,2,1,6,325,591,Recuento (Para recuento-SRA),I,05/06/24 21:00
4,'060202C0300','060202C03002',6,COLIMA,2,VALLE DE LAS GARZAS,202,3,C,0,...,1,3,2,0,11,304,591,Cotejo (Levantada en Casilla),I,05/06/24 12:09


In [39]:
import re

# --- 3. LIMPIEZA Y ESTRUCTURACIÓN ---
df_mapped = df.copy()

# a) Limpieza robusta de nombres de columna
def limpiar_nombre_columna(col_name):
    clean_col = col_name.lower()
    clean_col = re.sub(r'[/ ]', '_', clean_col)
    clean_col = re.sub(r'[^a-z0-9_]', '', clean_col)
    return clean_col.strip('_')
df_mapped.columns = [limpiar_nombre_columna(col) for col in df_mapped.columns]

# b) Creación de la columna 'casilla' estandarizada
if 'tipo_casilla' in df_mapped.columns and 'id_casilla' in df_mapped.columns:
    df_mapped['id_casilla'] = df_mapped['id_casilla'].astype(str)
    df_mapped['casilla_final'] = df_mapped['tipo_casilla'].str.strip() + df_mapped['id_casilla']
    print("     -> Columna 'casilla' estandarizada creada exitosamente.")
else:
    df_mapped['casilla_final'] = 'N/A'

# c) Eliminar columnas de contexto y redundantes
columnas_a_eliminar = [
    'clave_casilla', 'clave_acta', 'id_entidad', 'entidad', 'id_distrito_federal',
    'distrito_federal', 'ext_contigua', 'tipo_acta', 'observaciones',
    'mecanismos_traslado', 'fecha_hora', 'id_casilla', 'tipo_casilla', 'casilla'
]
df_mapped = df_mapped.drop(columns=columnas_a_eliminar, errors='ignore')

# d) Renombrar columnas
df_mapped = df_mapped.rename(columns={
    'casilla_final': 'casilla',
    'candidato_a_independiente': 'independiente',
    'candidato_a_no_registrado_a': 'voto_no_registrado',
    'votos_nulos': 'voto_nulo',
    'total_votos_calculados': 'voto_total'
})

# e) Corregir tipos de dato (reemplazar '-')
columnas_a_convertir = [
    'independiente', 'pan_pri_prd', 'pan_pri', 'pan_prd', 'pri_prd',
    'pvem_pt_morena', 'pvem_pt', 'pvem_morena', 'pt_morena'
]
for col in columnas_a_convertir:
    if col in df_mapped.columns:
        if df_mapped[col].dtype == 'object':
            df_mapped[col] = df_mapped[col].replace('-', '0', regex=False)
        df_mapped[col] = pd.to_numeric(df_mapped[col], errors='coerce')
df_mapped = df_mapped.fillna(0)

# --- 4. CÁLCULO DE MÉTRICAS ANALÍTICAS ---
competidores_2024 = [
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'morena', 'independiente',
    'pan_pri_prd', 'pan_pri', 'pan_prd', 'pri_prd',
    'pvem_pt_morena', 'pvem_pt', 'pvem_morena', 'pt_morena'
]
competidores_2024 = [col for col in competidores_2024 if col in df_mapped.columns]
partidos_morena_2024_total = [col for col in ['morena', 'pvem', 'pt', 'pvem_pt_morena', 'pvem_pt', 'pvem_morena', 'pt_morena'] if col in df_mapped.columns]
partidos_morena_2024_aliados = [col for col in ['pvem', 'pt', 'pvem_pt_morena', 'pvem_pt', 'pvem_morena', 'pt_morena'] if col in df_mapped.columns]
oposicion_2024 = [col for col in competidores_2024 if col not in partidos_morena_2024_total]

# El resto de la lógica es nuestro modelo estándar
df_mapped['morena_coalitions'] = df_mapped[partidos_morena_2024_total].sum(axis=1)
df_mapped['non_morena_coalitions'] = df_mapped[oposicion_2024].sum(axis=1)
df_mapped['voto_morena_solo'] = df_mapped['morena']
df_mapped['voto_morena_aliados'] = df_mapped[partidos_morena_2024_aliados].sum(axis=1)
df_competidores = df_mapped[competidores_2024]
df_mapped['ganador_casilla_partido'] = df_competidores.idxmax(axis=1)
df_mapped['ganador_casilla_votos'] = df_competidores.max(axis=1)
if len(df_competidores.columns) > 1:
    segundo_lugar_votos = np.sort(df_competidores.values, axis=1)[:, -2]
    df_mapped['segundo_lugar_votos'] = segundo_lugar_votos
    temp_df = df_competidores.copy()
    for index, row in temp_df.iterrows():
        winner_col = df_mapped.loc[index, 'ganador_casilla_partido']
        if pd.notna(winner_col) and winner_col in temp_df.columns:
            temp_df.loc[index, winner_col] = np.nan
    df_mapped['segundo_lugar_partido'] = temp_df.idxmax(axis=1)
    df_mapped['margen_victoria'] = df_mapped['ganador_casilla_votos'] - df_mapped['segundo_lugar_votos']
else:
    df_mapped['segundo_lugar_votos'] = 0
    df_mapped['segundo_lugar_partido'] = "N/A"
    df_mapped['margen_victoria'] = df_mapped['ganador_casilla_votos']

# --- 5. ORGANIZACIÓN FINAL Y GUARDADO ---
columnas_finales = [
    'seccion', 'casilla', 'lista_nominal',
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'morena', 'independiente',
    'pan_pri_prd', 'pan_pri', 'pan_prd', 'pri_prd',
    'pvem_pt_morena', 'pvem_pt', 'pvem_morena', 'pt_morena',
    'morena_coalitions', 'non_morena_coalitions', 'voto_morena_solo', 'voto_morena_aliados',
    'ganador_casilla_partido', 'ganador_casilla_votos', 'segundo_lugar_partido', 
    'segundo_lugar_votos', 'margen_victoria',
    'voto_no_registrado', 'voto_nulo', 'voto_total'
]
columnas_existentes = [col for col in columnas_finales if col in df_mapped.columns]
df_mapped = df_mapped[columnas_existentes]

print("\nDatos limpios de 2024 (Presidente) con el modelo de análisis completo:")
df_mapped.info()

# Guardar el archivo limpio
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, output_file)
df_mapped.to_csv(output_path, index=False)
print(f"\nArchivo limpio y actualizado de 2024 (Presidente) guardado en: {output_path}")

     -> Columna 'casilla' estandarizada creada exitosamente.

Datos limpios de 2024 (Presidente) con el modelo de análisis completo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   seccion                  217 non-null    int64 
 1   casilla                  217 non-null    object
 2   lista_nominal            217 non-null    int64 
 3   pan                      217 non-null    int64 
 4   pri                      217 non-null    int64 
 5   prd                      217 non-null    int64 
 6   pvem                     217 non-null    int64 
 7   pt                       217 non-null    int64 
 8   mc                       217 non-null    int64 
 9   morena                   217 non-null    int64 
 10  pan_pri_prd              217 non-null    int64 
 11  pan_pri                  217 non-null    int64 
 12  pan_prd        

## Base consolidada elecciones PRESIDENTE en Manzanillo 2012-2024

In [40]:

import glob

# --- 1. CONFIGURACIÓN ---
BASE_DIR = '/Users/omartellez/Manzanillo'

# Apuntamos a la carpeta de Presidente limpios
input_folder = os.path.join(BASE_DIR, '02_datos_limpios', 'Presidente') 
output_folder = os.path.join(BASE_DIR, '03_base_consolidada')
output_file = 'presidente_consolidado_2012_2024.csv'

# --- 2. ENCONTRAR Y CARGAR TODOS LOS ARCHIVOS LIMPIOS ---
search_pattern = os.path.join(input_folder, '*_limpio.csv')
clean_files = sorted(glob.glob(search_pattern))

if not clean_files:
    print("No se encontraron archivos limpios de Presidente para consolidar.")
else:
    all_dfs = []
    print(f"Se encontraron {len(clean_files)} archivos para consolidar:")
    for file_path in clean_files:
        print(f"  -> Cargando: {os.path.basename(file_path)}")
        
        # Extraer el año del nombre del archivo (ej. '..._2012_limpio.csv')
        try:
            year = int(os.path.basename(file_path).split('_')[-2])
        except (ValueError, IndexError):
            year = None

        df = pd.read_csv(file_path)
        df['anio'] = year
        df['tipo_eleccion'] = 'Presidente'
        
        all_dfs.append(df)

    # --- 3. CONSOLIDAR, ORDENAR Y GUARDAR ---
    consolidated_df = pd.concat(all_dfs, ignore_index=True)
    
    # Lógica de ordenamiento automático y robusto
    columnas_inicio = [
        'seccion', 'casilla', 'anio', 'tipo_eleccion', 'lista_nominal',
        'ganador_casilla_partido', 'ganador_casilla_votos', 'segundo_lugar_partido', 
        'segundo_lugar_votos', 'margen_victoria',
        'morena_coalitions', 'non_morena_coalitions', 'voto_morena_solo', 'voto_morena_aliados'
    ]
    columnas_fin = ['voto_no_registrado', 'voto_nulo', 'voto_total']
    
    columnas_votos_crudos = sorted([
        col for col in consolidated_df.columns 
        if col not in columnas_inicio and col not in columnas_fin
    ])
    
    orden_final = columnas_inicio + columnas_votos_crudos + columnas_fin
    
    consolidated_df_ordenado = consolidated_df[orden_final]
    consolidated_df_final = consolidated_df_ordenado.fillna(0)
        
    # Guardar la base de datos consolidada
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, output_file)
    consolidated_df_final.to_csv(output_path, index=False)
    
    print("\n¡Base de datos de Presidente consolidada y ordenada creada exitosamente!")
    print(f"Guardada en: {output_path}")
    print("\nResumen de la base consolidada final:")
    consolidated_df_final.info()

Se encontraron 3 archivos para consolidar:
  -> Cargando: Manzanillo_PRESIDENTE_2024_limpio.csv
  -> Cargando: Manzanillo_Presidente_2012_limpio.csv
  -> Cargando: Manzanillo_Presidente_2018_limpio.csv

¡Base de datos de Presidente consolidada y ordenada creada exitosamente!
Guardada en: /Users/omartellez/Manzanillo/03_base_consolidada/presidente_consolidado_2012_2024.csv

Resumen de la base consolidada final:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650 entries, 0 to 649
Data columns (total 49 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   seccion                  650 non-null    int64  
 1   casilla                  650 non-null    object 
 2   anio                     650 non-null    int64  
 3   tipo_eleccion            650 non-null    object 
 4   lista_nominal            650 non-null    int64  
 5   ganador_casilla_partido  650 non-null    object 
 6   ganador_casilla_votos    650 non-null    int